In [1]:
! pip install -e custom-diffusers/

Obtaining file:///home/tran.duc.trungb/triton/triton-playground/02.build_tensorrt_from_sd/custom-diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for custom-diffusers (pyproject.toml) ... done
  Created wheel for custom-diffusers: filename=custom_diffusers-0.0.1-py3-none-any.whl size=2381 sha256=22dbd451a5a38dcb800014cc03e7cb6668ed8fa64a4cb9dedd17fb7696263f5a
  Stored in directory: /tmp/pip-ephem-wheel-cache-hq395s7y/wheels/51/d5/5f/828313c9b55687deaff022c888c213d0f9ab7dc88d696d5f37
Successfully built custom-diffusers
  Attempting uninstall: custom-diffusers
    Found existing installation: custom-diffusers 0.0.1
    Uninstalling custom-diffusers-0.0.1:
      Successfully uninstalled custom-diffusers-0.0.1


In [2]:
from custom_diffusers import UNet2DConditionModel
from diffusers import DiffusionPipeline
from optimum.exporters.onnx import export
from optimum.onnxruntime import ORTStableDiffusionPipeline
from pathlib import Path
import onnx
import torch

/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Download model

In [7]:
TORCH_DIR = Path("./exported-models/torch")

TORCH_DIR.mkdir(parents=True, exist_ok=True)


ONNX_DIR = Path("./exported-models/onnx")

ONNX_DIR.mkdir(parents=True, exist_ok=True)

In [5]:
MODEL_ID = "runwayml/stable-diffusion-v1-5"


In [6]:
pipe = DiffusionPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    use_safetensors=True,
)
pipe = pipe.to("cuda")

prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt).images[0]


pipe.save_pretrained(TORCH_DIR / "stable-diffusion-v1-5")

Loading pipeline components...:  14%|█▍        | 1/7 [00:00<00:01,  5.34it/s]

library_name transformers
library_name diffusers


Loading pipeline components...:  29%|██▊       | 2/7 [00:00<00:02,  2.38it/s]

library_name diffusers
library_name stable_diffusion


Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  5.44it/s]


library_name transformers
library_name diffusers
library_name transformers


100%|██████████| 50/50 [00:02<00:00, 18.45it/s]


# Export models to ONNX

In [8]:
SAVE_PATH = ONNX_DIR / "stable-diffusion-v1-5"

if not SAVE_PATH.exists():
    try:

        SAVE_PATH.mkdir(parents=True)
        pipeline = ORTStableDiffusionPipeline.from_pretrained(MODEL_ID, export=True)
        pipeline.save_pretrained(SAVE_PATH.as_posix())
    except Exception as e:
        print(e)
        SAVE_PATH.rmdir()

# else:
#     pipeline = ORTStableDiffusionPipeline.from_pretrained(
#         SAVE_PATH.as_posix(),
#     )

Framework not specified. Using pt to export the model.
Keyword arguments {'subfolder': '', 'use_auth_token': None, 'trust_remote_code': False} are not expected by StableDiffusionPipeline and will be ignored.
Loading pipeline components...:  14%|█▍        | 1/7 [00:00<00:00,  6.43it/s]

library_name transformers
library_name diffusers


Loading pipeline components...:  43%|████▎     | 3/7 [00:00<00:00, 10.96it/s]

library_name diffusers
library_name stable_diffusion


Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 13.07it/s]

library_name transformers
library_name diffusers
library_name transformers




***** Exporting submodel 1/4: CLIPTextModel *****
Using framework PyTorch: 2.3.0+cu121
/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/transformers/modeling_attn_mask_utils.py:86: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1 or self.sliding_window is not None:
/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/transformers/modeling_attn_mask_utils.py:162: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:
/home/tran.duc.trungb/mi

In [ ]:
# prompt = "sailing ship in storm by Leonardo da Vinci"
# image = pipeline(prompt, num_inference_steps=2).images[0]

# pipeline.save_pretrained("./models/onnx/stable-diffusion-v1-5")

In [ ]:

# pipe = DiffusionPipeline.from_pretrained(
#     "./torch-stable-diffusion-v1-5",
#     torch_dtype=torch.float16,
#     use_safetensors=True,
# )
# pipe = pipe.to("cuda")

# prompt = "a photo of an astronaut riding a horse on mars"
# image = pipe(prompt).images[0]



# Export Unet

In [ ]:
unet = UNet2DConditionModel.from_pretrained("./exported-models/torch/stable-diffusion-v1-5/unet")

In [ ]:
timesteps = torch.tensor([1, 2]).reshape(-1, 1).float()

latent_model_input = torch.randn(2, 4, 64, 64)

prompt_embeds = torch.randn(2, 77, 768)

_ = unet(timestep=timesteps, sample=latent_model_input, encoder_hidden_states=prompt_embeds)

In [ ]:
from custom_diffusers.config import UNetOnnxConfig

In [ ]:
onnx_config = UNetOnnxConfig(unet.config)

In [ ]:
onnx_path = Path("./exported-models/onnx/unet.onnx")

onnx_inputs, onnx_outputs = export(unet, onnx_config, onnx_path, onnx_config.DEFAULT_ONNX_OPSET)

In [ ]:
onnx.checker.check_model(onnx_path.as_posix())

In [ ]:
from optimum.exporters.onnx import validate_model_outputs

validate_model_outputs(
    onnx_config, unet, onnx_path, onnx_outputs, onnx_config.ATOL_FOR_VALIDATION
)